In [1]:
import pandas as pd
import os


#read files
file_path = "../Input_Files/California CBSA 37100 January 2025 Medicare Part B ASC Fee Schedule Excel File.xlsx"
df = pd.read_excel(file_path, skiprows=6)

print(df)

     HCPCS / CPT CODE MODIFIER PROCEDURE INDICATOR   AMOUNT  \
0               A9527      NaN                   A   208.58   
1               C1600      NaN                   C     0.00   
2               C1601      NaN                   C     0.00   
3               C1602      NaN                   C     0.00   
4               C1603      NaN                   C     0.00   
...               ...      ...                 ...      ...   
4191            93459       TC                   S  1857.04   
4192            93460       TC                   S  1857.04   
4193            93461       TC                   S  1857.04   
4194            93985       TC                   S   146.12   
4195            93986       TC                   S    63.61   

      FC HCPCS MODIFIER AMOUNT (mod FC)  FB HCPCS MODIFIER AMOUNT(mod FB)  \
0                                   0.0                               0.0   
1                                   0.0                               0.0   
2           

In [2]:
import re
from datetime import datetime

# File name
filename = "California CBSA 37100 January 2025 Medicare Part B ASC Fee Schedule Excel File.xlsx"

# Month mapping
month_map = {
    'January': '01', 'February': '02', 'March': '03',
    'April': '04', 'May': '05', 'June': '06',
    'July': '07', 'August': '08', 'September': '09',
    'October': '10', 'November': '11', 'December': '12'
}

# Extract year and month
year_match = re.search(r'\b(20\d{2})\b', filename)
month_match = next((month for month in month_map if month in filename), None)

if year_match and month_match:
    year = year_match.group(1)
    month = month_map[month_match]
    day = '01'
    full_date = f"{year}-{month}-{day}"
    
    print("day:", day)
    print("month:", month)
    print("year:", year)
    print("date:", full_date)
else:
    print("Could not extract date info.")
# Create a directory named after the date if it doesn't exist
if not os.path.exists(f'../Output_Files/{year}/{full_date}'):
    os.makedirs(f'../Output_Files/{year}/{full_date}')    

day: 01
month: 01
year: 2025
date: 2025-01-01


In [3]:
#df.columns = ['SVC_CD', 'Modifier_1','Procedure_Indicator' 'FEE','FC_HCPCS_MODIFIER_AMOUNT','FB_HCPCS_MODIFIER_AMOUNT', 'PENALTY_FEE','FC_MODIFIER_PENALTY_AMOUNT','FB_MODIFIER_PENALTY_AMOUNT','PAYMENT_CAP_PRICE']
df.columns = [
    'HCPCS_CPT_CODE',
    'MODIFIER',
    'PROCEDURE_INDICATOR',
    'AMOUNT',
    'FC_HCPCS_MODIFIER_AMOUNT_mod_FC',
    'FB_HCPCS_MODIFIER_AMOUNT_mod_FB',
    'PENALTY_FEE_AMOUNT',
    'FC_MODIFIER_PENALTY_PRICE',
    'FB_MODIFIER_PENALTY_PRICE',
    'PAYMENT_CAP_PRICE'
]

print(df)

     HCPCS_CPT_CODE MODIFIER PROCEDURE_INDICATOR   AMOUNT  \
0             A9527      NaN                   A   208.58   
1             C1600      NaN                   C     0.00   
2             C1601      NaN                   C     0.00   
3             C1602      NaN                   C     0.00   
4             C1603      NaN                   C     0.00   
...             ...      ...                 ...      ...   
4191          93459       TC                   S  1857.04   
4192          93460       TC                   S  1857.04   
4193          93461       TC                   S  1857.04   
4194          93985       TC                   S   146.12   
4195          93986       TC                   S    63.61   

      FC_HCPCS_MODIFIER_AMOUNT_mod_FC  FB_HCPCS_MODIFIER_AMOUNT_mod_FB  \
0                                 0.0                              0.0   
1                                 0.0                              0.0   
2                                 0.0        

In [7]:
# 1. Filter out rows where all relevant amount columns are zero or null
df_filtered = df[
    (df['AMOUNT'] != 0) | 
    (df['FC_HCPCS_MODIFIER_AMOUNT_mod_FC'] != 0) | 
    (df['FB_HCPCS_MODIFIER_AMOUNT_mod_FB'] != 0)
].copy()

# Initialize an empty list to hold transformed rows
transformed_rows = []

# 2. A and D without modifier
df_no_modifier = df_filtered[['HCPCS_CPT_CODE', 'AMOUNT']].copy()
df_no_modifier = df_no_modifier[df_no_modifier['AMOUNT'] != 0]
df_no_modifier['MODIFIER'] = ''
df_no_modifier.rename(columns={'AMOUNT': 'FEE'}, inplace=True)
transformed_rows.append(df_no_modifier)

# 3. A and E with "FC" modifier
df_fc_modifier = df_filtered[['HCPCS_CPT_CODE', 'FC_HCPCS_MODIFIER_AMOUNT_mod_FC']].copy()
df_fc_modifier = df_fc_modifier[df_fc_modifier['FC_HCPCS_MODIFIER_AMOUNT_mod_FC'] != 0]
df_fc_modifier['MODIFIER'] = 'FC'
df_fc_modifier.rename(columns={'FC_HCPCS_MODIFIER_AMOUNT_mod_FC': 'FEE'}, inplace=True)
transformed_rows.append(df_fc_modifier)

# 4. A and F with "FB" modifier
df_fb_modifier = df_filtered[['HCPCS_CPT_CODE', 'FB_HCPCS_MODIFIER_AMOUNT_mod_FB']].copy()
df_fb_modifier = df_fb_modifier[df_fb_modifier['FB_HCPCS_MODIFIER_AMOUNT_mod_FB'] != 0]
df_fb_modifier['MODIFIER'] = 'FB'
df_fb_modifier.rename(columns={'FB_HCPCS_MODIFIER_AMOUNT_mod_FB': 'FEE'}, inplace=True)
transformed_rows.append(df_fb_modifier)

# Combine all the pieces
df_final = pd.concat(transformed_rows, ignore_index=True)

# Optional: reorder columns
df_final = df_final[['HCPCS_CPT_CODE', 'MODIFIER', 'FEE']]
df_final = df_final.rename(columns={
    'HCPCS_CPT_CODE': 'SVC_CD',
    'MODIFIER': 'MODIFIER_1',
    'FEE': 'FEE'
})
print(df_final)

#df_final.to_csv(f'../Output_Files/{year}/{full_date}/MCare_ASC.{full_date}.txt', sep='|', index=False)

print("file saved")

     SVC_CD MODIFIER_1       FEE
0     A9527               208.58
1     C1716               868.33
2     C1717               342.39
3     C1719               564.50
4     C2616             17485.10
...     ...        ...       ...
5173  69729         FB   2728.94
5174  69730         FB   3922.30
5175  69930         FB   4393.36
5176  92920         FB   1651.35
5177  92928         FB   3238.95

[5178 rows x 3 columns]
file saved


In [10]:
# 1. Filter out rows where all relevant amount columns are zero or null
df_filtered = df[
    (df['AMOUNT'] != 0) | 
    (df['FC_HCPCS_MODIFIER_AMOUNT_mod_FC'] != 0) | 
    (df['FB_HCPCS_MODIFIER_AMOUNT_mod_FB'] != 0)
].copy()

# Initialize an empty list to hold transformed rows
transformed_rows = []

# 2. A and D with modifier from column B
df_with_modifier = df_filtered[['HCPCS_CPT_CODE', 'AMOUNT', 'MODIFIER']].copy()
df_with_modifier = df_with_modifier[df_with_modifier['AMOUNT'] != 0]
df_with_modifier.rename(columns={'AMOUNT': 'FEE', 'MODIFIER': 'MODIFIER'}, inplace=True)
transformed_rows.append(df_with_modifier)

# 3. A and E with "FC" modifier
df_fc_modifier = df_filtered[['HCPCS_CPT_CODE', 'FC_HCPCS_MODIFIER_AMOUNT_mod_FC']].copy()
df_fc_modifier = df_fc_modifier[df_fc_modifier['FC_HCPCS_MODIFIER_AMOUNT_mod_FC'] != 0]
df_fc_modifier['MODIFIER'] = 'FC'
df_fc_modifier.rename(columns={'FC_HCPCS_MODIFIER_AMOUNT_mod_FC': 'FEE'}, inplace=True)
transformed_rows.append(df_fc_modifier)

# 4. A and F with "FB" modifier
df_fb_modifier = df_filtered[['HCPCS_CPT_CODE', 'FB_HCPCS_MODIFIER_AMOUNT_mod_FB']].copy()
df_fb_modifier = df_fb_modifier[df_fb_modifier['FB_HCPCS_MODIFIER_AMOUNT_mod_FB'] != 0]
df_fb_modifier['MODIFIER'] = 'FB'
df_fb_modifier.rename(columns={'FB_HCPCS_MODIFIER_AMOUNT_mod_FB': 'FEE'}, inplace=True)
transformed_rows.append(df_fb_modifier)

# Combine all the pieces
df_final = pd.concat(transformed_rows, ignore_index=True)

# Optional: reorder and rename columns
df_final = df_final[['HCPCS_CPT_CODE', 'MODIFIER', 'FEE']]
df_final = df_final.rename(columns={
    'HCPCS_CPT_CODE': 'SVC_CD',
    'MODIFIER': 'MODIFIER_1',
    'FEE': 'FEE'
})

print(df_final)

# Uncomment to save file
df_final.to_csv(f'../Output_Files/{year}/{full_date}/MCare_ASC.{full_date}.txt', sep='|', index=False)

print("file saved")


     SVC_CD MODIFIER_1       FEE
0     A9527        NaN    208.58
1     C1716        NaN    868.33
2     C1717        NaN    342.39
3     C1719        NaN    564.50
4     C2616        NaN  17485.10
...     ...        ...       ...
5173  69729         FB   2728.94
5174  69730         FB   3922.30
5175  69930         FB   4393.36
5176  92920         FB   1651.35
5177  92928         FB   3238.95

[5178 rows x 3 columns]
file saved
